# Load necessaries

In [ ]:
%pip install transformers==4.44.0

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
if torch.cuda.is_available():
    device = 'cuda'
else:
    raise Exception('No GPU available')
import numpy as np

In [3]:
system_prompt = {
'basic' :"""请仔细阅读以下多条文本,然后回答问题：基于这些文本，这个用户是否存在抑郁情绪？用“是”或者“否”回答
***重要***
你必须用以下格式回答问题
答案：“是”或“否”
""",
'OR': """请仔细阅读以下多条文本，关注用户的情绪表达、对未来的态度、社交参与度、兴趣水平以及任何关于身体症状或睡眠/食欲变化、自我价值、个人感受的表述。回答问题：基于这些文本，这个用户是否存在抑郁情绪？用“是”或者“否”回答，再给出你一步一步的分析过程。
***重要***
你必须用以下格式回答问题
答案：“是”或“否”
分析：你一步一步的分析过程，解释你为什么给出这个答案。""",
'OR2': """请仔细阅读以下多条文本然后回答问题：基于这些文本，这个用户是否存在抑郁情绪？用“是”或者“否”回答，再给出你一步一步的分析过程。
***重要***
你必须用以下格式回答问题
答案：“是”或“否”
分析：你一步一步的分析过程，解释你为什么给出这个答案。""",
'OR3': """请仔细阅读以下多条文本然后回答问题：基于这些文本，这个用户是否存在抑郁情绪？用“是”或者“否”回答，再给出你一步一步的分析过程。
""",

'RET':"""请逐条阅读以下用户的文本，关注用户的情绪表达、对未来的态度、社交参与度、兴趣水平以及任何关于身体症状或睡眠/食欲变化、自我价值、个人感受的表述。回答问题：基于这些文本，这个用户是否存在抑郁情绪？用“是”或者“否”回答，再给出你一步一步的分析过程，指出哪条或哪几条文本让你得出这个回答。
***重要***
如果用户在文本中明确提到自己得到了抑郁的诊断，请直接判断为“是”，并在分析中指出相关文本。
你必须用以下格式回答问题
答案：“是”或“否”
分析：你一步一步的分析过程，指出哪条或哪几条文本让你得出这个回答。""",}

In [4]:
def modify_tweet (tweet,mode = 1):
    input_text_list = tweet.split('\n')
    if mode == 0 :
        tweet = tweet
    elif mode == 1:
        input_text_list_mod= [f'文本{i+1}:{text}' for i,text in enumerate(input_text_list)]
        tweet = '\n'.join(input_text_list_mod)
    elif mode == 2:
        input_text_list_mod = [f'<文本{i+1}>{text}</文本{i+1}> ' for i,text in enumerate(input_text_list)]
        tweet = '\n'.join(input_text_list_mod)
    return tweet
# RMSNorm normalization
def rmsnorm(x, eps=1e-6): #x: Input tensor，eps: Small constant to prevent division by zero，return: Normalized tensor
    rms = torch.sqrt(torch.mean(x ** 2, dim=-1, keepdim=True) + eps)
    return x / rms

def find_start_end_index(start_token = '<|im_start|>',end_token ='<|im_end|>',drift = 3,glm = False):
    input_start = 0
    input_end = 0
    output_start = 0
    output_end = 0
    input_count = 0
    output_count = 0
    i = 0
    for token in generated_ids[0][0]:
        if glm :
            token = [token]
        if tokenizer.decode(token) == start_token:
            if input_count == 0:
                input_start = i+drift
                input_count += 1
            else:
                output_start = i+drift
        elif tokenizer.decode(token) == end_token:
            if output_count == 0:
                input_end = i
                output_count += 1
            else:
                output_end = i
        i += 1
    print('input_start:',input_start,'input_end:',input_end)
    print('output_start:',output_start,'output_end:',output_end)
    assert input_start < input_end
    #assert output_start < output_end
    return input_start,input_end,output_start,output_end

def compute_attention_rollout(converted_attentions,response,norm_type = 'RMS',add_residual=True, eps=1e-6):
    all_attentions =  [(t,a) for t,a in zip(response.tolist(), converted_attentions)]
    attention_scores =[]
    for pair in all_attentions:
        token = pair[0]
        att_mat = pair[1].mean(axis=1) #Calculate the average attention head weights to obtain a tensor of shape [layer, seq_len or 1, seq_lenor or seq_lenor + generation steps]

        if add_residual:
            att_mat = att_mat + np.eye(att_mat.shape[1])[None,...] #residual connection
        if norm_type == 'RMS':
            att_mat = rmsnorm(torch.tensor(att_mat), eps=eps).numpy() #RMSNorm normalization
        elif norm_type == 'layernorm':
            att_mat = att_mat / att_mat.sum(axis=-1)[...,None] #Layer normalization
        att_mat = rmsnorm(torch.tensor(att_mat), eps=eps).numpy()
        joint_att = np.zeros(att_mat.shape)
        layers = joint_att.shape[0]
        joint_att[0] = att_mat[0]
        for i in np.arange(1, layers):
            joint_att[i] = att_mat[i].dot(joint_att[i-1].T)
        attention_scores.append((token,joint_att))
    return attention_scores
# Generate an HTML file based on the input vector and text, where each character displays its corresponding score above it.
# Adjust the color based on the score. Save the final HTML
# Parameters:
# tensor (numpy.ndarray): A vector of shape (1, N), where N is the length of the text.
# text (str): A text string matching the length of the vector.
# output_path (str): The path to save the output HTML file.
# normalize (bool): Whether to normalize the scores, default is True.
# method (str): Normalization method, supports 'min-max' (default), 'mean', and 'moving-average'.
# window_size (int): Window size for moving average, default is 10, used only when moving average normalization is selected.

# Returns:
# None
def generate_text_with_scores_html(tensor, text, output_path, normalize=True, method='min-max', window_size=10):
    
    # Check the shape of the tensor and the length of the text
    assert tensor.shape[1] == len(text), "Tensor length must match the text length."

    scores = tensor[0]

    # Normalize the scores
    if normalize:
        if method == 'min-max':
            scores = (scores - scores.min()) / (scores.max() - scores.min())
        elif method == 'mean':
            mean_value = scores.mean()
            scores = scores - mean_value
        elif method == 'moving-average':
            mean_value = np.convolve(scores, np.ones(window_size) / window_size, mode='same')
            scores = scores - mean_value
        elif method == 'z-score':
            mean_value = scores.mean()
            std_value = scores.std()
            scores = (scores - mean_value) / std_value
        else:
            raise ValueError("Unknown normalization method! Please choose from 'min-max', 'mean', 'moving-average', or 'z-score'.")

    # Convert the score to color
    def score_to_color(score):
        r = int(255 * score)
        b = 255 - r
        return f'rgb({r}, 0, {b})'

    # Calculate the number of lines
    chars_per_line = 50
    num_lines = len(text) // chars_per_line + (1 if len(text) % chars_per_line else 0)

    # Generate the HTML content
    html_content = "<html><body style='font-family:monospace;'>\n"

    # Add the text with scores
    spacing = "20px"  # Adjust the spacing between characters

    for line in range(num_lines):
        start_idx = line * chars_per_line
        end_idx = start_idx + chars_per_line
        line_text = text[start_idx:end_idx]
        line_scores = scores[start_idx:end_idx]

        for i, char in enumerate(line_text):
            color = score_to_color(line_scores[i])
            score_text = f"{line_scores[i]:.2f}"
            border_style = "border: 1px solid black;" if line_scores[i] > 0 else ""
            html_content += f"<div style='display:inline-block; text-align:center; color:{color}; margin-right:{spacing}; {border_style}'>" \
                            f"<div style='font-size:0.5em;'>{score_text}</div>" \
                            f"<div>{char}</div>" \
                            f"</div>"

        html_content += "<br>\n"

    html_content += "</body></html>"

    # Save the HTML content to a file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"HTML Generated at {output_path}")

# Load tweet data

In [5]:
import json
with open ('datas_positive.json','r',encoding='utf-8') as f:
  datas_positive = json.load(f)
with open ('datas_negative.json','r',encoding='utf-8') as f:
  datas_negative = json.load(f)

In [6]:
tweet_sample = datas_positive['user_42']

# Generate the responses

## Llama

In [4]:
model_id = "shenzhi-wang/Llama3-8B-Chinese-Chat"
#model_id = "D:\LLMs\Llama3-8B-Chinese-Chat"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype= torch.bfloat16, device_map="auto",attn_implementation="eager", 
                                             )

In [12]:
prompt = system_prompt['OR2']+'\n'+modify_tweet(tweet_sample,mode=0)
prompt_length = len(tokenizer(system_prompt['OR2'])['input_ids'])
input_ids = tokenizer.apply_chat_template(
    [{"role": "system", "content": prompt}],
    add_generation_prompt=True,
    return_tensors="pt"
    ).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

model_inputs = tokenizer.apply_chat_template(
    [{"role": "system", "content": prompt}],
    add_generation_prompt=True,
    return_tensors="pt"
    ).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
generate_params_llama = {
                    # Change this based on how much vram you have,in our experiments, we set it to 350
                    # so the final attention rollout may vary if you set it to a different value
                     'max_new_tokens': 100, 
                     'pad_token_id': tokenizer.pad_token_id,
                     #'terminator_ids': terminators,
                     'do_sample': False,
                     'output_attentions': True,
                     'return_dict_in_generate': True,
        }

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        **generate_params_llama
        )
response_ids = generated_ids[0][0][input_ids.shape[-1]:]
response = tokenizer.decode(response_ids, skip_special_tokens=False)
print(response)
start_index,end_index,_,_ = find_start_end_index(start_token = '<|end_header_id|>',end_token ='<|eot_id|>',drift = 2)

## QWEN

In [7]:
model_id = "Qwen/Qwen2-7B-Instruct"
#model_id = "D:\LLMs\Qwen2-7B-Instruct"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
qwen_generate_params = {
    # Change this based on how much vram you have,in our experiments, we set it to 350
    # so the final attention rollout may vary if you set it to a different value
    "max_new_tokens": 100, 
    "output_attentions": True,
    "return_dict_in_generate": True,
    "do_sample": False,
}
prompt = system_prompt['OR2']+'\n'+modify_tweet(tweet_sample,mode=0)
prompt_length = len(tokenizer(system_prompt['OR2'])['input_ids'])
messages = [
    {"role": "system", "content":prompt },
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
with torch.no_grad():
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(model_inputs.input_ids, **qwen_generate_params)
response_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids[0])]
response = tokenizer.batch_decode(response_ids, skip_special_tokens=False)[0]
print(response)
start_index,end_index,_,_ = find_start_end_index(drift=2)

## GLM

In [7]:
#model_id = "THUDM/glm-4-9b-chat"
model_id = "D:\LLMs\glm-4-9b-chat"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager" ,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

### Modify the generation code so that it returns attentions matrix

In [ ]:
#This is a must for glm model,sinice GLM use its own custom generation codes (modeling_chatglm.py)
#Which dose not output attentions by default
import os
import shutil


cache_dir = os.path.expanduser("~/.cache/huggingface/hub/")
target_file = "modeling_chatglm.py"


def find_file(directory, filename):
    for root, dirs, files in os.walk(directory):
        if filename in files:
            return os.path.join(root, filename)
    return None

file_path = find_file(cache_dir, target_file)

if file_path:
    print(f"Found: {file_path}")
    model_path = os.path.dirname(file_path)
    backup_file = os.path.join(model_path, "modeling_chatglm.py.bak")
    os.rename(file_path, backup_file)
    print(f"Renamed original file to: {backup_file}")
    script_file = "modeling_chatglm.py"
    destination = os.path.join(model_path, script_file)
    shutil.copy(script_file, destination)
    print(f"Copied new script to: {destination}")   
else:
    print("File not found.")

In [ ]:
prompt = system_prompt['OR2']+'\n'+modify_tweet(tweet_sample,mode=0)
prompt_length = len(tokenizer(system_prompt['OR2'])['input_ids'])
inputs = tokenizer.apply_chat_template([{"role": "system", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to(model.device)
generated_ids = model.generate(
    **inputs,
    # Change this based on how much vram you have,in our experiments, we set it to 350
    # so the final attention rollout may vary if you set it to a different value
    max_new_tokens=100,
    do_sample = False,
    output_attentions=True,
    return_dict_in_generate=True,
)
response_ids = generated_ids[0][:, inputs['input_ids'].shape[1]:]
print(tokenizer.decode(response_ids[0], skip_special_tokens=False))
start_index,end_index,_,_ = find_start_end_index(start_token='<|system|>',end_token='<|assistant|>',drift=2,glm = True)

# Rollout Calculations


In [10]:
alpha = 0.2
beta = 0.8

## Self-attention Rollout (Input seq)

In [ ]:
s = start_index
t = end_index
attention_input = [
    torch.max(hd[:, :, s:t, s:t].cpu().squeeze(0).to(torch.float32), dim=0)[0]
    for hd in generated_ids['attentions'][0]
]
print(len(attention_input))
print(attention_input[0].shape)

In [ ]:
reviced_attention = []
attender_attention = []
recived_pairs = []
index_tokens = [ tokenizer.decode(t,skip_special_tokens=False) for t in generated_ids[0][0][s:t]]
seq_len = len(index_tokens)
for layer in attention_input:
    total_attention_as_object = torch.sum(layer, axis=0).tolist()
    total_attention_as_object = [att/seq_len for att in total_attention_as_object]
    total_attention_as_attender = torch.sum(layer, axis=1).tolist()
    total_attention_as_attender = [att/seq_len for att in total_attention_as_attender]
    reviced_attention.append(total_attention_as_object)
    attender_attention.append(total_attention_as_attender)
i = 1
for layer in reviced_attention:
    recived_pairs.append([(token,att) for token,att in zip(index_tokens,layer)])
    print(f'layer{i}:{sorted(recived_pairs[-1],key=lambda x:x[1],reverse=True)[:10]}')
    i += 1


In [13]:
reviced_attention_tensor = [np.array(layer) for layer in reviced_attention]
bias_vector = np.ones(reviced_attention_tensor[0].shape[0])
bias_vector = bias_vector[None, :]
reviced_attention_tensor = np.array(reviced_attention_tensor) + bias_vector
reviced_attention_tensor = rmsnorm(torch.tensor(reviced_attention_tensor)).numpy()   
layers = reviced_attention_tensor.shape[0]
attention_score = np.zeros(reviced_attention_tensor.shape)
attention_score[0] = reviced_attention_tensor[0]
for i in np.arange(1, layers):
    # Perform element-wise multiplication. The original formula is used for calculating an attention matrix [seq_len, seq_len],
    # but here it's applied to an attention vector [1, seq_len], so element-wise multiplication is required.
    attention_score[i] = reviced_attention_tensor[i] * attention_score[i-1]  
last_layer_self_rollout = attention_score[-1] 

## Reforme the output attention matrix

In [ ]:
print('Gen Step:',type(generated_ids['attentions']),len(generated_ids['attentions']))
print('Layers in each step：',type(generated_ids['attentions'][0]),len(generated_ids['attentions'][0]))
print('Att matrix in each step:',type(generated_ids['attentions'][0][0]),generated_ids['attentions'][0][0].shape)
# Applicable to attention weights without shape transformation, formatted as shown above.
if 'llama' in model_id.lower():
    all_attentions =  [(t,a) for t,a in zip(response_ids.tolist(), generated_ids['attentions'])] #attention dict for all tokens
    all_attentions_avg = [(t,sum(a)/len(a)) for t,a in zip(response_ids.tolist(), generated_ids['attentions'])] #average attention dict for all tokens
elif 'qwen' in model_id.lower():
    
    all_attentions =  [(t,a) for t,a in zip(response_ids[0].tolist(), generated_ids['attentions'])]
    all_attentions_avg = [(t,sum(a)/len(a)) for t,a in zip(response_ids[0].tolist(), generated_ids['attentions'])] #average attention dict for all tokens
print(all_attentions_avg[0][1].shape)
for i in range(len(generated_ids['attentions'])):
    for j in range(len(generated_ids['attentions'][i])):
        print(f'Gen Step {i+1},Layer {j+1}',generated_ids['attentions'][i][j].shape)

### Reforme the 1st step

In [ ]:
attentions_step1 = generated_ids['attentions'][0]

# Create a list to store the last token attention matrix for each layer
last_token_attention_list = []
assert generated_ids[0][0][end_index:end_index+1] == tokenizer.eos_token_id
# Iterate through each layer's attention matrix
for layer_attention in attentions_step1:
    # Choose the last token's attention weights
    last_token_attention = layer_attention[:, :, end_index:end_index+1:, :]
    last_token_attention_list.append(last_token_attention)


# Show the shape of the last token attention matrix for the first layer
print(last_token_attention_list[0].shape)
print(len(last_token_attention_list))
attentions_list = list(generated_ids['attentions'])

# Convert the list of last token attention matrices to a tuple
attentions_list[0] = tuple(last_token_attention_list)

# Convert the list back to a tuple and assign it to generated_ids['attentions']
generated_ids['attentions'] = tuple(attentions_list)

# Verify the shape of the attention matrix for the first layer
print(type(generated_ids['attentions']))  # Shoud be tuple
print(generated_ids['attentions'][0][0].shape)  # Should be [batch_size, attention_head, 1, seq_len+1]


# Create a list to store the converted attention matrices
converted_attentions = []
#这里的start_index和end_index代表的是输入序列的起始和结束位置

for i in range(len(generated_ids['attentions'])):
    # Concatenate the tensors from 28 layers in each generation step, each with shape 
    # [batch_size = 1, attention_head , seq_len or 1, seq_len or seq_len + generation steps], 
    # into a single tensor with shape [layer, attention_head, seq_len or 1, seq_len or seq_len + generation steps].
    step_attentions = torch.cat([layer for layer in generated_ids['attentions'][i]], dim=0)
    # Retain the content between start_index and end_index
    if step_attentions.shape[-1] > start_index:
        step_attentions = step_attentions[:, :, :, start_index:end_index]
    
    converted_attentions.append(step_attentions.detach().clone().cpu().to(torch.float32).numpy())

# Print the shape of the converted attention matrices
# Should be [layer, attention_head, seq_len or 1, seq_len or seq_len + generation steps]
for i, tensor in enumerate(converted_attentions):
    print(f"Shape of tensor at step {i+1}: {tensor.shape}")

## Attention Rollout (Gene steps)

In [16]:
eps = 1e-6
if 'llama' in model_id.lower():
    all_attentions = [(t, a) for t, a in zip(response_ids.tolist(), converted_attentions)]
elif 'qwen' in model_id.lower():
    all_attentions =  [(t,a) for t,a in zip(response_ids[0].tolist(), converted_attentions)]
attention_scores = []
for pair in all_attentions:
    token = pair[0]
    # att_mat = pair[1].mean(axis=1)  # Average attention head weights, resulting in a tensor of shape [layer=28, seq_len=input length or 1, seq_len=input length or input length + generation steps]
    att_mat = pair[1].max(axis=1)  # Take the maximum attention head weights, resulting in a tensor of shape [layer=28, seq_len=input length or 1, seq_len=input length or input length + generation steps]
    bias_vector = np.ones(pair[1].shape[-1])
    bias_vector = bias_vector[None, None, :]
    att_mat = att_mat + bias_vector  # Apply residual connection on [layers, 1, seq_len]
    att_mat = rmsnorm(torch.tensor(att_mat), eps=eps).numpy()  # Apply RMSNorm normalization
    # att_mat = att_mat / att_mat.sum(axis=-1)[..., None]  # Normalize
    joint_att = np.zeros(att_mat.shape)
    layers = joint_att.shape[0]
    joint_att[0] = att_mat[0]
    for i in np.arange(1, layers):
        # Perform element-wise multiplication. The original formula is for calculating an attention matrix [seq_len, seq_len]
        # but here it's applied to an attention vector [1, seq_len], so element-wise multiplication is needed
        joint_att[i] = att_mat[i] * joint_att[i-1]  
    attention_scores.append((token, joint_att))

#### Calculate the last layer attention across all layers and output to HTML

In [ ]:
seq_ids = []
if 'llama' in model_id.lower():
    seq_ids = (model_inputs[0].tolist())[start_index:end_index]
elif 'qwen' in model_id.lower():
    seq_ids = (model_inputs['input_ids'].tolist())[0][start_index:end_index]
seq_tokens = [tokenizer.decode([token]) for token in seq_ids][prompt_length+1:] # Extract the input tokens
all_generated_attention_scores_avg = np.array([att[1][:,:,prompt_length+1:] for att in attention_scores]).mean(axis=0) # Calculate the average step attention scores 
last_layer = all_generated_attention_scores_avg[-1] # Extract the last layer attention scores
last_layer_self_rollout = last_layer_self_rollout.reshape(1,-1)[:,prompt_length+1:]
last_layer = (alpha * last_layer) + (beta* last_layer_self_rollout)
print(last_layer.mean())
print(len(seq_tokens))
generate_text_with_scores_html(last_layer,seq_tokens,normalize= True,method='z-score',output_path=f'TEST.html')

#### Cumulative Attention Animation (Work in Progress)

In [18]:
cumulative_avg_attentions = []

def list_add(a,b):
    return [a[i]+b[i] for i in range(len(a))]

# Accumulate the average attention step by step for each generation step
for step in range(len(attention_scores)):
    # Take the attention from the first step up to step+1
    avg_attention = np.array([att[1][:,:,prompt_length+1:] for att in attention_scores[:step+1]]).mean(axis=0)[-1] * beta # multiply by beta
    # Add the cumulative average attention of the current step to the list
    cumulative_avg_attentions.append(avg_attention)

last_layer_self_rollout_ = last_layer_self_rollout.reshape(1, -1)[:,prompt_length+1:]
last_layer_self_rollout_ = alpha * last_layer_self_rollout # multiply by alpha

#z-score normalization
cumulative_avg_attentions = [(att - att.mean()/att.std()).tolist()[0] for att in cumulative_avg_attentions]
self_rollout_mean = last_layer_self_rollout_.mean()
self_rollout_std = last_layer_self_rollout_.std()
last_layer_self_rollout_ = (last_layer_self_rollout_ - self_rollout_mean)/self_rollout_std
last_layer_self_rollout_ = last_layer_self_rollout_.tolist()[0]
#cumulative_avg_attentions = [list_add(att,last_layer_self_rollout_) for att in cumulative_avg_attentions]


if 'llama' in model_id.lower():
    response_per_token = [tokenizer.decode([token]) for token in response_ids]
elif 'qwen' in model_id.lower():
    response_per_token = [tokenizer.decode([token]) for token in response_ids[0]]
with open('attention_data.js', 'w') as f:
    f.write(f"const seq_tokens = {json.dumps(seq_tokens)};\n")
    f.write(f"const cumulative_avg_attentions = {json.dumps(cumulative_avg_attentions)};\n")
    f.write(f"const response_per_token = {json.dumps(response_per_token)};\n")
